In [24]:
# Прочитаем датасет и выведем первые пять его строк:

import pandas as pd

melb_df = pd.read_csv('data/melb_data_fe.csv')
melb_df.head()

# Давайте посмотрим на информацию о столбцах с помощью метода info():

display(melb_df.info())

# Преобразуем столбец Date в формат datetime и выделим квартал (quarter) продажи объектов недвижимости.
# Найдём второй по популярности квартал продажи

melb_df['Date'] = pd.to_datetime(melb_df['Date'])
quarters = melb_df['Date'].dt.quarter
print(quarters.value_counts().iloc[1])

# Преобразуем все столбцы, в которых меньше 150 уникальных значений, в тип данных category,
# исключив из преобразования столбцы Date, Rooms, Bedroom, Bathroom, Car.

cols_to_exclude = ['Date', 'Rooms', 'Bedroom', 'Bathroom', 'Car'] 
max_unique_count = 150 
for col in melb_df.columns: 
    if melb_df[col].nunique() < max_unique_count and col not in cols_to_exclude: 
        melb_df[col] = melb_df[col].astype('category')
display(melb_df.info())

# МЕТОД SORT_VALUES()

# СОРТИРОВКА ПО ЗНАЧЕНИЯМ ОДНОГО СТОЛБЦА

# Отсортируем таблицу по возрастанию цены объектов недвижимости (Price):

melb_df.sort_values(by='Price').head(10)

# А теперь отсортируем таблицу по убыванию (от самой последней до самой первой) даты продажи объекта (Date). 
# Для этого выставим параметр ascending на False:

melb_df.sort_values(by='Date', ascending=False)

# СОРТИРОВКА ПО ЗНАЧЕНИЯМ НЕСКОЛЬКИХ СТОЛБЦОВ

# Для сортировки по значениям нескольких столбцов необходимо передать названия этих столбцов в параметр by в виде списка.
# При этом важно обращать внимание на порядок следования столбцов.

# отсортируем таблицу сначала по возрастанию расстояния от центра города (Distance),
# а затем — по возрастанию цены объекта (Price). Для того чтобы вывод был более наглядным,
# выделим каждую десятую строку из столбцов Distance и Price результирующей таблицы:

melb_df.sort_values(by=['Distance', 'Price']).loc[::10, ['Distance', 'Price']]

# Мы получили таблицу, отсортированную по возрастанию расстояния до центра города. 
# Если встречаются объекты недвижимости, у которых расстояние оказывается одинаковым, 
# то внутри такой группы производится сортировка по цене объекта.

# КОМБИНИРОВАНИЕ СОРТИРОВКИ С ФИЛЬТРАЦИЕЙ

# Предположим, компания McGrath поручила нам восстановить хронологию продаж таунхаусов,
# у которых площадь участка существенно больше площади здания, чтобы понять,
# как часто компания справляется с таким сложным видом объектов. Объекты, проданные в один и тот же день,
# мы бы хотели сортировать по значению коэффициента соотношения площадей.

# Найдём информацию о таунхаусах (Type), проданных компанией (SellerG) McGrath,
# у которых коэффициент соотношения площадей здания и участка (AreaRatio) меньше -0.8.
# Результат отсортируем по дате продажи (Date) в порядке возрастания,
# а после проведём сортировку по убыванию коэффициента соотношения площадей. Также обновим старые индексы на новые,
# установив параметр ignore_index на True. Для наглядности результата выберем из таблицы только столбцы Data и AreaRatio: 

mask1 = melb_df['AreaRatio'] < -0.8
mask2 = melb_df['Type'] == 'townhouse'
mask3 = melb_df['SellerG'] == 'McGrath'
melb_df[mask1 & mask2 & mask3].sort_values(
    by=['Date', 'AreaRatio'],
    ascending=[True, False],
    ignore_index=True
).loc[:, ['Date', 'AreaRatio']]

# Отсортируем таблицу по столбцу AreaRatio с параметром ascending=False (по убыванию).
# Выставим параметр ignore_index на True, чтобы заменить индексы на новые.
# С помощью метода loc извлечем значение столбца BuildingArea в строке с индексом 1558.

int(melb_df.sort_values(
    by='AreaRatio', 
    ignore_index=True,
    ascending=False
).loc[1558, 'BuildingArea'])

# Произведем фильтрацию таблицы и отберите строки по условиям «тип здания — таунхаус» и «число жилых комнат больше 2».
# Отсортируем полученную таблицу по столбцу Rooms с параметром ascending=True (по возрастанию)
# и по средней площади комнат с параметром ascending=False (по убыванию).
# Выставим параметр ignore_index на True, чтобы заменить индексы на новые.
# С помощью метода loc извлечем значение столбца Price в строке с индексом 18.

mask1 = melb_df['Type'] == 'townhouse'
mask2 = melb_df['Rooms'] > 2
int(melb_df[mask1&mask2].sort_values(
    by=['Rooms', 'MeanRoomsSquare'],
    ascending=[True, False],
    ignore_index=True
).loc[18, 'Price'])

# Группировка данных в DataFrame

# Одна из основных задач анализа данных — это группировка данных и сравнение показателей в группах.
# Например, нам необходимо сравнить средний уровень заработной платы в зависимости от пола/уровня образования.
# Или же мы хотим проследить, какая группа клиентов приносит нам наибольший доход, чтобы направить своё внимание на эту группу.

# МЕТОД GROUPBY()

# by — имя или список имён столбцов, по которым производится группировка.
# axis — ось, по которой производится группировка (0 — строки, 1 — столбцы). По умолчанию группировка производится по строкам.
# as_index — добавляется ли дополнительный индекс к таблице. По умолчанию установлен на True.

#ГРУППИРОВКА ДАННЫХ ПО ОДНОМУ КРИТЕРИЮ С ОДНОЙ АГРЕГАЦИЕЙ

# Применим агрегирующую функцию среднего к результату работы groupby().
# В качестве столбца для группировки возьмём столбец типа объекта недвижимости (Type):

melb_df.groupby(by='Type').mean()

# Как правило, нам не нужна информация обо всех столбцах, поэтому агрегирующие методы можно применять только к интересующему нас столбцу.
# Например, давайте сравним средние цены на объекты в зависимости от их типа:

melb_df.groupby('Type')['Price'].mean()

# Из этой маленькой таблицы видно, что наибольшей средней ценой обладают объекты типа house (дома, коттеджи, виллы).
# Следовательно, можно сделать вывод, что тип постройки является значимым фактором при определении цены объекта недвижимости.

# Теперь давайте выясним, какие регионы (Regionname) наиболее удалены от центра Мельбурна.

# Для этого найдём минимальное значение расстояния от центра города до объекта в зависимости от его региона.
# Результат отсортируем по убыванию расстояния:

melb_df.groupby('Regionname')['Distance'].min().sort_values(ascending=False)

# Итак, наиболее удалёнными являются все регионы Victoria.

# ГРУППИРОВКА ДАННЫХ ПО ОДНОМУ КРИТЕРИЮ С НЕСКОЛЬКИМИ АГРЕГАЦИЯМИ

# Чтобы рассчитать несколько агрегирующих методов, можно воспользоваться методом agg(),
# который принимает список строк с названиями агрегаций.

# Давайте построим таблицу для анализа продаж по месяцам. Для этого найдём количество продаж,
# а также среднее и максимальное значения цен объектов недвижимости (Price), сгруппированных по номеру месяца продажи (MonthSale).
# Результат отсортируем по количеству продаж в порядке убывания:

melb_df.groupby('MonthSale')['Price'].agg(
    ['count', 'mean', 'max']
).sort_values(by='count', ascending=False)

# так как мы считаем несколько показателей для одного столбца, в результате мы получаем объект DataFrame.

# В результате применения метода agg(), в который мы передали список с названиями интересующих нас агрегирующих функций,
# мы получаем DataFrame со столбцами count, mean и max, где для каждого месяца рассчитаны соответствующие параметры.
# Результат сортируем по столбцу count.

# Какие интересные выводы можно сделать из этой таблицы?

# 1 - Пик продаж приходится на период весна-лето.
# 2 - Средняя цена продаваемых объектов относительно стабильна и находится в пределах 1 млн. австралийских долларов
#     с небольшими отклонениями (около 100 тыс. влево и вправо).
# 3 - Прослеживается некоторая зависимость между сезоном и максимальной ценой объектов: в месяцы с большим спросом
#     на объекты недвижимости цена также имеет наибольшие показатели. Можно сделать предположение, что это связано с повышением
#     цен на элитные дома в периоды большого спроса.

# Если нам нужна полная информация обо всех основных статистических характеристиках внутри каждой группы,
# Мы можем воспользоваться методом agg(), передав в качестве его параметра строку 'describe':

melb_df.groupby('MonthSale')['Price'].agg('describe')

# После базовых математических функций наиболее частым агрегированием является подсчёт числа уникальных значений.
# Так, например, мы можем вычислить число уникальных риелторских компаний в зависимости от региона, чтобы понять,
# в каких регионах конкуренция на рынке недвижимости меньше. Это можно сделать, передав в параметр метода agg() строку 'nunique'. 

# Более того, метод agg() поддерживает использование и других функций. Передадим дополнительно встроенную функцию set,
# чтобы получить множество из агентств недвижимости, которые работают в каждом из регионов:

melb_df.groupby('Regionname')['SellerG'].agg(['nunique', set])

# Как и ожидалось, наименьшая конкуренция в наиболее удалённом регионе Western Victoria,
# а наибольшая — в центральном районе Northern Metropolitan.

# Сгруппируем данные по признаку количества комнат и найдите среднюю цену объектов недвижимости в каждой группе.

melb_df.groupby('Rooms')['Price'].mean().sort_values(ascending=False)

# Определим какой регион имеет наименьшее стандартное отклонение по географической широте (Lattitude).

melb_df.groupby('Regionname')['Lattitude'].std().sort_values()

# Определим какая риелторская компания (SellerG) имеет наименьшую общую выручку за период с 1 мая по 1 сентября (включительно) 2017 года?
# Для ответа на этот вопрос рассчитаем сумму продаж (Price) каждой компании в заданный период. Не забудем перевести даты в формат datetime.

date1 = pd.to_datetime('2017-05-01')
date2 = pd.to_datetime('2017-09-01')
mask = (date1 <= melb_df['Date']) & (melb_df['Date']<= date2)
melb_df[mask].groupby('SellerG')['Price'].sum().sort_values(ascending=True)

# Сводные таблицы

# Сводные таблицы — это распространённый инструмент для агрегации данных.
#  Сводная таблица принимает на вход данные из отдельных столбцов и группирует их. В результате получается новая таблица,
# которая позволяет увидеть многомерное обобщение данных. Таким образом,
# благодаря сводным таблицам мы можем оценить зависимость между двумя и более признаками данных.

# МЕТОД GROUPBY КАК СПОСОБ ПОСТРОЕНИЯ СВОДНЫХ ТАБЛИЦ

# Мы можем строить таблицу, которая показывает зависимость медианной цены и площади здания от числа комнат:

melb_df.groupby('Rooms')[['Price', 'BuildingArea']].median()

# Также можно построить таблицу, в которой мы будем учитывать не только число комнат, но и тип здания (Type).
# Для этого в параметрах метода groupby() укажем список из нескольких интересующих нас столбцов.

melb_df.groupby(['Rooms', 'Type'])['Price'].mean()

# В результате выполнения такого кода мы получаем Series, которая обладает несколькими уровнями индексов:
# первый уровень — число комнат, второй уровень — тип здания. Такая организация индексов называется иерархической.
# Вычисление параметра (средней цены) происходит во всех возможных комбинациях признаков.

# Для того, чтобы финальный результат был представлен в виде сводной таблицы (первый группировочный признак по строкам,
# а второй — по столбцам), а не в виде Series с иерархическими индексами, к результату чаще всего применяют метод unstack(),
# который позволяет переопределить вложенный индекс в виде столбцов таблицы:

melb_df.groupby(['Rooms', 'Type'])['Price'].mean().unstack()

# В результате мы получаем сводную таблицу, столбцы в которой представляют типы домов (house, townhouse, unit),
# строки — число комнат, а на пересечении строк и столбцов находится средняя стоимость объекта с такими показателями.

# Какие интересные выводы можно сделать из этой таблицы?
# 1. Пропуски в сводной таблице (NaN) говорят о том, что в наших данных нет соответствующих комбинаций признаков.
#    Например, у нас нет информации о ценах на таунхаусы, где количество комнат больше пяти.
# 2. Наибольшей средней стоимостью (2,25 млн. австралийских долларов) обладают объекты типа unit с восемью жилыми комнатами.
#    Наименьшая средняя стоимость — у однокомнатных домов типа unit (чуть меньше 400 тыс. австралийских долларов).
# 3. Сколько бы комнат ни было в доме, цена на объекты типа unit всегда ниже других (за исключением восьмикомнатных объектов).

# МЕТОД PIVOT_TABLE ДЛЯ ПОСТРОЕНИЯ СВОДНЫХ ТАБЛИЦ

# На самом деле метод groupby редко используется при двух параметрах, так как для построения сводных таблиц существует специальный
# и более простой метод — pivot_table().

#  Основные параметры метода pivot_table()
# values — имя столбца, по которому необходимо получить сводные данные, применяя агрегирующую функцию;
# index — имя столбца, значения которого станут строками сводной таблицы;
# columns — имя столбца, значения которого станут столбцами сводной таблицы;
# aggfunc — имя или список имён агрегирующих функций (по умолчанию — подсчёт среднего, 'mean');
# fill_value — значение, которым необходимо заполнить пропуски (по умолчанию пропуски не заполняются).

# Давайте построим ту же самую таблицу, но уже с использованием метода pivot_table.
# В качестве параметра values укажем столбец Price, в качестве индексов сводной таблицы возьмём Rooms,
# а в качестве столбцов — Type. Агрегирующую функцию оставим по умолчанию (среднее).
# Дополнительно заменим пропуски в таблице на значение 0. 
# Финальный результат для наглядности вывода округлим с помощью метода round() до целых.

melb_df.pivot_table(
    values='Price',
    index='Rooms',
    columns='Type',
    fill_value=0
).round()

# А теперь давайте проанализируем продажи в каждом из регионов в зависимости от того,
# будний был день или выходной. Для этого построим сводную таблицу,
# в которой строками будут являться названия регионов (Regionname),
# а в столбцах будет располагаться наш «признак-мигалка» выходного дня (Weekend), который равен 1, если день был выходным,
# и 0 — в противном случае. В качестве значений сводной таблицы возьмём количество продаж.

melb_df.pivot_table(
    values='Price',
    index='Regionname',
    columns='Weekend',
    aggfunc='count'
)

# Из результирующей таблицы можно сделать два вывода:
# 1. Число продаж резко возрастает в выходные вне зависимости от региона (приблизительно в 2-3 раза).
#    То есть вероятность того, что дом продадут в выходные, гораздо выше вероятности, что его продадут в будний день
# 2. В отдалённых регионах (Victoria) коэффициент роста числа продаж выше, чем в центральных.
#    Если в центральных регионах Metropolitan продажи по выходным в 2-2.5 раза выше, чем по будням,
#    то в регионах Victoria число продаж в выходные вырастает примерно в 3 раза.

# Такой рост можно даже попытаться объяснить логически: в выходные дни у людей появляется свободное время,
# чтобы доехать до отдалённых пригородов с целью покупки дома.

# Разберём ещё один пример: найдём, как зависит средняя и медианная площадь участка (Landsize) от типа объекта (Type)
# и его региона (Regionname). Чтобы посмотреть несколько статистических параметров, нужно передать в аргумент aggfunc список
# из агрегирующих функций. Построим такую сводную таблицу, где пропущенные значения заменим на 0:

melb_df.pivot_table(
    values='Landsize',
    index='Regionname',
    columns='Type',
    aggfunc=['median', 'mean'],
    fill_value=0
)

# Обратм внимание на добавление дополнительных индексов столбцов median и mean.
# Здесь медианное и среднее значения рассчитаны отдельно для каждой комбинации признаков.
# Здесь в глаза бросаются объекты типа house в регионах Eastern Victoria и Northern Victoria
# — в них среднее и медиана отличаются более чем в три раза. Вероятно, это связано с тем,
# что в этих районах очень большой разброс цен: есть несколько объектов с гигантской площадью,
# а остальные объекты имеют небольшую площадь. Из-за этого среднее значение искажается,
# в то время как медиана нечувствительна к такому разбросу и не искажает результат.

# МНОГОМЕРНЫЕ СВОДНЫЕ ТАБЛИЦЫ

# Для того чтобы исследовать зависимость от большего числа признаков,
# можно передать список признаков в параметр index или параметр columns.

# Давайте построим таблицу, в которой по индексам будут располагаться признаки метода продажи (Method) и типа объекта (Type),
# по столбцам — наименование региона (Regionname), а на пересечении строк и столбцов будет стоять медианная цена объекта (Price):

melb_df.pivot_table(
    values='Price',
    index=['Method','Type'],
    columns='Regionname',
    aggfunc='median',
    fill_value=0
)

# Первым индексом в таблице идёт метод продажи здания, далее для метода указывается тип недвижимости.
# По столбцам расположены регионы. В ячейках таблицы указана медианная цена для каждой такой комбинации.

# Такие таблицы уже сложнее читать, однако с помощью них можно более глубоко исследовать закономерности.
# Например, можно видеть, что вне зависимости от метода продажи и региона цена на объекты типа house практически всегда выше,
# чем на объекты другого типа.

# ДОСТУП К ДАННЫМ В СВОДНОЙ ТАБЛИЦЕ

# Давайте рассмотрим, что собой представляют столбцы сложной сводной таблицы.

# Запишем сводную таблицу, которую мы создавали ранее в переменную pivot:
# Выведем её столбцы с помощью атрибута columns:

pivot = melb_df.pivot_table(
    values='Landsize',
    index='Regionname',
    columns='Type',
    aggfunc=['median', 'mean'],
    fill_value=0
)
pivot.columns

# В результате мы получаем объект MultiIndex. Этот объект хранит в себе шесть комбинаций пар столбцов
# (два статистических параметра и три типа здания), то есть есть шесть возможных вариантов обращения к столбцам таблицы.

# Мультииндексы раскрываются подобно вложенным словарям — по очереди, как матрёшка. Чтобы получить доступ к определённому столбцу,
# вы должны сначала обратиться к столбцу, который находится уровнем выше.

# Так, из таблицы pivot мы можем получить средние значения площадей участков для типа здания unit,
# просто последовательно обратившись по имени столбцов:

display(pivot['mean']['unit'])

# Аналогично производится и фильтрация данных. Например, если нам нужны регионы,
# в которых средняя площадь здания для домов типа house меньше их медианной площади, то мы можем найти их следующим образом:

mask = pivot['mean']['house'] < pivot['median']['house']
filtered_pivot = pivot[mask]
display(filtered_pivot)

# Чтобы получить индексы отфильтрованной таблицы, можно воспользоваться атрибутом index и обернуть результат в список:

print(list(filtered_pivot.index))
# ['Southern Metropolitan', 'Western Metropolitan']

# Составим сводную таблицу, которая показывает зависимость медианной площади (BuildingArea) 
# здания от типа объекта недвижимости (Type) и количества жилых комнат в доме (Rooms).

pivot = melb_df.pivot_table(
    values='BuildingArea',
    index='Type',
    columns='Rooms',
    aggfunc='median',
    fill_value=0
)
display(pivot)

# Составим сводную таблицу, которая показывает зависимость медианной цены объекта недвижимости (Price)
# от риелторского агентства (SellerG) и типа здания (Type).
# Во вновь созданной таблице найдем агентство, у которого медианная цена для зданий типа unit максимальна.

pivot = melb_df.pivot_table(
    values='Price',
    index='SellerG',
    columns='Type',
    aggfunc='median',
)
max_unit_price = pivot['unit'].max()
print(pivot[pivot['unit'] == max_unit_price].index[0])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Suburb           13580 non-null  object 
 1   Rooms            13580 non-null  int64  
 2   Type             13580 non-null  object 
 3   Price            13580 non-null  float64
 4   Method           13580 non-null  object 
 5   SellerG          13580 non-null  object 
 6   Date             13580 non-null  object 
 7   Distance         13580 non-null  float64
 8   Postcode         13580 non-null  int64  
 9   Bedroom          13580 non-null  int64  
 10  Bathroom         13580 non-null  int64  
 11  Car              13580 non-null  int64  
 12  Landsize         13580 non-null  float64
 13  BuildingArea     13580 non-null  float64
 14  CouncilArea      12211 non-null  object 
 15  Lattitude        13580 non-null  float64
 16  Longtitude       13580 non-null  float64
 17  Regionname  

None

4359
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Suburb           13580 non-null  category      
 1   Rooms            13580 non-null  int64         
 2   Type             13580 non-null  category      
 3   Price            13580 non-null  float64       
 4   Method           13580 non-null  category      
 5   SellerG          13580 non-null  category      
 6   Date             13580 non-null  datetime64[ns]
 7   Distance         13580 non-null  float64       
 8   Postcode         13580 non-null  int64         
 9   Bedroom          13580 non-null  int64         
 10  Bathroom         13580 non-null  int64         
 11  Car              13580 non-null  int64         
 12  Landsize         13580 non-null  float64       
 13  BuildingArea     13580 non-null  float64       
 14  CouncilArea      12211 non-null  

None

C:\Users\Konstantin\AppData\Local\Temp\ipykernel_33796\2591673883.py:120: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  melb_df.groupby(by='Type').mean()


Regionname
Eastern Metropolitan          330.444444
Eastern Victoria              295.333333
Northern Metropolitan         495.026538
Northern Victoria               0.000000
South-Eastern Metropolitan    357.864865
Southern Metropolitan         466.380245
Western Metropolitan          557.637232
Western Victoria                0.000000
Name: unit, dtype: float64

median                       mean              \
Type                   house townhouse unit       house   townhouse   
Regionname                                                            
Southern Metropolitan  586.0     246.0    0  569.643881  278.858824   
Western Metropolitan   531.0     198.0   62  507.883406  244.560669   

                                   
Type                         unit  
Regionname                         
Southern Metropolitan  466.380245  
Western Metropolitan   557.637232

['Southern Metropolitan', 'Western Metropolitan']


Rooms,1,2,3,4,5,6,7,8,10
Type,,,,,,,,,
house,126.0,126,126,141.0,177,126,216.5,126,126
townhouse,88.0,114,126,159.5,152,0,0.0,0,0
unit,69.5,110,126,126.0,0,171,0.0,126,0


Nick
